# STA 141B Lecture 9

The class website is <https://github.com/2019-winter-ucdavis-sta141b/notes>

### Announcements

### Topics

* Undocumented APIs
* XML and HTML
* Web Scraping

### Datasets

* [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)
* [Wikipedia's List of Largest Cities](https://en.wikipedia.org/wiki/List_of_largest_cities)
* [CUESA's Vegetable Seasons Chart](https://cuesa.org/eat-seasonally/charts/vegetables)

### References

* [__requests__ documentation](http://docs.python-requests.org/en/master/)
* [__requests-html__ documentation](https://html.python-requests.org/)
* [MDN HTML Reference](https://developer.mozilla.org/en-US/docs/Web/HTML/Element)
* [XPath Diner](http://www.topswagcode.com/xpath/) -- an interactive XPath tutorial
* [CSS Diner](https://flukeout.github.io/) -- an interactive CSS Selector tutorial
* Python for Data Analysis, Ch. 6
* Python for Data Analysis, Ch. 7.3 (to review string processing)

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/

## Getting Data from the Web

Revised list of ways you can get data from the web, from most to least convenient:

1. Direct download or "data dump"
2. Python or R package (there are packages for many popular web APIs)
3. Documented web API
4. Undocumented web API
5. Scraping

## Undocumented Web APIs

Many websites use undocumented web APIs to get data. For example:

* [University of California Compensation](https://ucannualwage.ucop.edu/wage/)
* [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

You can identify these websites by looking at requests in your browser's developer tools. In Firefox or Chrome, you can open the developer tools with `ctrl-shift-i`.

Requests to web APIs almost always return JSON or XML data. By examining the browser requests, you can work out the endpoints and parameters, allowing you to use the API.

**CAUTION:** Web APIs that are undocumented are often undocumented for a reason. Using an undocumented API may make someone angry or get you into legal trouble! Government and quasi-government websites (like the examples above) are probably okay, as long as you cache and rate-limit your requests. For everything else, find for an alternative or get permission first.

Let's reverse engineer the Yolo County Health Inspections web API so that we can get data about local restaurants.

In [1]:
import numpy as np
import pandas as pd
import requests
import requests_cache

requests_cache.install_cache("mycache")

In [12]:
def get_health_info(q):
    response = requests.post("https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities", params = {
        "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4",
    }, json = {
        "FacilityName": q
    })

    response.raise_for_status()
    #"FacilityName": "pluto's"

    # Different ways to attach data to a POST request:
    # With data=, we get a query string
    #   FacilityName=pluto's

    # With json=, we get a json object

    return response.json()

get_health_info("pluto's")
get_health_info("yakitori")

[{'FacilityId': 'FA0016377',
  'FacilityName': 'YAKITORI YUCHAN',
  'Address': '109 E ST ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': '63958585-5175-4387-a88f-a94200ecc3aa'}]

We could reverse engineer other parts of the API to get detailed data about health violations.

## Web Scraping

### What makes a web page?

Web pages are written in _hypertext markup language_ (HTML). HTML files (`.htm` or `.html`) are plain text, just like JSON, Python scripts, and R scripts.

In HTML, we use _tags_ to create _elements_ of a web page. Elements add formatting and structure to the page.

* Tags usually come in pairs: an opening tag and a closing tag.
* Tags are written `<NAME>` for opening tags, `</NAME>` for closing tags, and `<NAME />` for singleton tags.
* Opening and singleton tags can have _attributes_ that contain additional information. Attributes are written `ATTRIBUTE=VALUE` after the tag name. 

See [here](https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/HTML_basics) for a more detailed explanation, and [here](https://developer.mozilla.org/en-US/docs/Web/HTML/Element) for a list of valid HTML elements.

#### Examples

As an example:

```html
<p>This <a href="https://www.youtube.com/watch?v=dQw4w9WgXcQ">page</a> is famous and this <strong>word</strong> is emphasized.</p>
```
The `p` tag marks a paragraph, the `a` tag marks a link (an _anchor_), and the `strong` tag marks emphasized text.

Here's a string that contains HTML for a simple, complete website:

In [14]:
page = """
<html>
<head>
    <title>This is the Title!</title>
</head>

<body>
    <p>This is a paragraph!</p>
    <p id="best-paragraph">This is another paragraph! 🌮</p>
    <p>Visit <a href="https://pudding.cool/">The Pudding</a>.</p>
    <span>This is a span. It comes with an avocado. 🥑</span>
</body>
</html>
"""

_Extensible markup language_ (XML) also uses tags to create elements. We say XML is _extensible_ because you can create your own XML elements (unlike HTML). People typically use XML to describe structure and meaning of data, rather than for formatting.

We'll use the same process to extract data from both HTML and XML.

### Helper Packages

A _parser_ converts formatted data into familiar data structures. We've used __requests__' built-in JSON parser, but the package doesn't have a built-in HTML/XML parser. Fortunately, there are many other Python packages for parsing HTML/XML and web scraping.

HTML/XML Parsers:
* [lxml](https://lxml.de/)
* [html5lib](https://github.com/html5lib/html5lib-python)
* [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/)
* [requests-html](https://html.python-requests.org/)

Scraper Frameworks (_convenient after learning the basics with parsers_):
* [scrapy](https://scrapy.org/)
* [newspaper3k](https://github.com/codelucas/newspaper)

Even more [here](https://github.com/lorien/awesome-web-scraping/blob/master/python.md#web-scraping-frameworks).

We'll use __lxml__ here, but you're welcome to use other packages on assignments and the project. To install __lxml__ for Anaconda, run `conda install -c anaconda lxml` in a shell.

In [15]:
import lxml.html as lx

html = lx.fromstring(page)
html

<Element html at 0x7f860a635048>

### Finding Elements

Elements are nested, so an HTML document is like a tree:
```
html
├── head
│   └── title
└── body
    ├── p
    ├── p
    ├── p
    │   └── a
    └── span
```
This is similar to the file system on your computer. The key difference is that elements at the same level can have the same tag name.

#### XPath

The _XML Path Language_ (XPath) lets us write paths to elements. XPath paths look a lot like file paths. XPath is not Python-specific!

The `.xpath()` method gets all elements at an XPath path:

In [16]:
html.xpath("/html/head/title")

[<Element title at 0x7f8609c4a868>]

In [17]:
html.xpath("/html/body/p")

[<Element p at 0x7f8609c4aa98>,
 <Element p at 0x7f8609c4aae8>,
 <Element p at 0x7f8609c4ab38>]

Since there may be more than one element, the method always returns a list.

Absolute paths are not robust for scraping. An update to a web page that adds a single tag can break a scraper that uses absolute paths. In XPath, `//` means "anywhere below". We'll use `//` often because it's more robust:

In [20]:
html.xpath("/html/body//a")

[<Element a at 0x7f8609c4ab88>]

What if we just elements that satisfy a certain condition? In XPath, `[ ]` filters out elements that don't match a condition. For example:

In [22]:
html.xpath("//p[@id = 'best-paragraph']")

[<Element p at 0x7f8609c4aae8>]

[XPath Diner](http://www.topswagcode.com/xpath/) is an interactive tutorial that teaches most of the XPath syntax. It takes about 20-60 minutes. Work through it to become an XPath ninja!

#### CSS Selectors

_Cascading Style Sheets_ (CSS) is another language for formatting elements in an HTML document. CSS provides another way to select elements, called _CSS selectors_.

CSS selectors are more concise but less flexible than XPath paths. The `.cssselect()` method gets all elements at a CSS selector:

In [23]:
html.cssselect("p")

[<Element p at 0x7f8609c4aa98>,
 <Element p at 0x7f8609c4aae8>,
 <Element p at 0x7f8609c4ab38>]

### Extracting Text and Attributes

There are two ways to get text from an element:

* `.text` gives text inside the element, but not its children
* `.text_content()` gives text inside the element and its children, with all tags removed

In [27]:
p = html.xpath("//p[@id = 'best-paragraph']")[0]
p.text_content()

'This is another paragraph! 🌮'

In [28]:
html.text

'\n'

In [29]:
html.text_content()

'\n\n    This is the Title!\n\n\n\n    This is a paragraph!\n    This is another paragraph! 🌮\n    Visit The Pudding.\n    This is a span. It comes with an avocado. 🥑\n\n'

We can get values from attributes on an element with `.attrib`, which is a dictionary:

In [32]:
p.attrib["id"]

'best-paragraph'

In [36]:
[x.attrib["href"] for x in html.xpath("//a")]

['https://pudding.cool/']

### Example: Scraping Tables

For data in a `table` element, we can use __Pandas__ instead of writing a scraper.

Wikipedia provides lots of useful information in tables. Let's get the Wikipedia list of [the world's largest cities][wiki].

[wiki]: https://en.wikipedia.org/wiki/List_of_largest_cities

In [63]:
# This example shows all of the steps to clean the data.
# In lecture we only did lines 4-5.

tabs = pd.read_html("https://en.wikipedia.org/wiki/List_of_largest_cities")
tbl = tabs[1]

def strip_footnote(x):
    """This function removes bracketed footnotes, such as '[1]'."""
    if pd.isna(x):
        return x
    
    return x.partition("[")[0]

# Combine table headers into one row.
cols = list(tbl.columns.get_level_values(0))
cols[-3:] = tbl.columns.get_level_values(1)[-3:]
tbl.columns = [strip_footnote(c).title() for c in cols]

# We don't need the Image column
tbl = tbl.drop(columns = "Image")

# Strip footnotes from the rest of the data
tbl = tbl.applymap(strip_footnote)

# Convert numeric columns to numeric dtypes
cols = tbl.columns[-3:]
tbl[cols] = tbl[cols].apply(lambda x: pd.to_numeric(x.str.replace(",", "")))
tbl

,City,Nation,City Proper,Metropolitan Area,Urban Area
0,Chongqing,China,30751600.0,17000000.0,8165500.0
1,Shanghai,China,24256800.0,24750000.0,23416000.0
2,Beijing,China,21516000.0,24900000.0,21009000.0
3,Lagos,Nigeria,16060303.0,21000000.0,13123000.0
4,Dhaka,Bangladesh,14399000.0,20000000.0,19580000.0
5,Mumbai,India,12478447.0,12771200.0,20748395.0
6,Chengdu,China,16044700.0,10376000.0,6316922.0
7,Karachi,Pakistan,14910352.0,NaN,NaN
8,Guangzhou,China,14043500.0,25000000.0,20800654.0
9,Istanbul,Turkey,14025000.0,13520000.0,14657000.0


### Writing Scrapers

When the data we want isn't in a `table` element, we have to write our own scraper.

The workflow for writing a scraper is the same regardless of the language you use:

1. Download pages with an HTTP request (usually `GET`)
2. Parse pages to extract text
3. Clean up extracted text with string methods or regex
4. Save cleaned results

### Example: CUESA Vegetable Seasons



CUESA (Center for Urban Education about Sustainable Agriculture) provides [a chart](https://cuesa.org/eat-seasonally/charts/vegetables) that shows when vegetables are in season. Let's scrape the chart.

In [45]:
result = pd.read_html("https://cuesa.org/eat-seasonally/charts/vegetables")

In [61]:
# Download the page
response = requests.get("https://cuesa.org/eat-seasonally/charts/vegetables")
response.raise_for_status()

# Parse the HTML
html = lx.fromstring(response.text)
html

# Find the table with the vegetable seasons
tab = html.xpath("//table")[0]

rows = tab.xpath(".//tr")
rows = rows[1:]

# Extract the header
header = rows[0]

header = [x.text for x in header.xpath(".//li")]

# To be continued in the next lecture...